In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
import numpy as np

In [2]:
file = 'dataset_test.csv'

dataset = pd.read_csv(file)
print(dataset.shape)
# dataset.head().transpose()


""" 전처리 """
cols_drop_info = ['종목코드', '기준가1', '녹인가1', '환매일 종가', '평가기준가']
cols_drop_duplicate = ['평가구분', '상환조건달성'] + ['녹인발생차수'] # unique 값이 1개
cols_drop_future = ['상환구분', '상환실현차수']

cols_drop_dt = ['발행일', '상환일', '평가시작일', '평가종료일', '환매결정일', '녹인발생일']
# 데이터 타입 변경
for col in cols_drop_dt:
    dataset[col] = pd.to_datetime(dataset[col])


# 피처로 쓰기 애매함
cols_cnt = [
    '녹인일수', '녹인일수_전', '영업일수', '상환일수'
]

cols_feature = ['차수', '기초자산개수', '녹인발생차수_차이']
cols_cat = ['환매일종가위치'] # 범주형 변수
cols_dummy = ['환매일종가위치_code']
dataset['환매일종가위치_code'] = dataset['환매일종가위치'].astype('category').cat.codes.astype(float)
cols_pct100 = [
    '상환조건(%)', '하한 수준(%)', '상환조건감소량(%)_prev', '상환조건감소량(%)_next',
    '환매일 수준(%)', '녹인대비상환수준(%)', '환매대비상환수준(%)', '환매대비상환수준(%)_next'
]
cols_pct = [
    '녹인비율', '녹인비율_전', 'H총증감률', 'H평균증감률', 'H일평균증감률', 'H이전대비증감률', '상환비율'
]
# 비율 단위 변경
for col in cols_pct:
    col_new = col+"(%)"
    dataset[col_new] = dataset[col]*100
    cols_pct100.append(col_new)

#
cols_drop = cols_drop_info + cols_drop_duplicate + cols_drop_future + cols_drop_dt + cols_cat

(104, 38)


In [3]:
# dataset['환매결정일'] = pd.to_datetime(dataset['환매결정일'])
# dataset.set_index('환매결정일', inplace= True)
dataset.head()

,종목코드,차수,평가종료일,상환조건(%),평가시작일,환매결정일,발행일,상환일,상환실현차수,녹인발생일,...,환매대비상환수준(%),환매대비상환수준(%)_next,환매일종가위치_code,녹인비율(%),녹인비율_전(%),H총증감률(%),H평균증감률(%),H일평균증감률(%),H이전대비증감률(%),상환비율(%)
0,KR6DS0000428,3,2022-07-13,85.0,2022-01-14,2022-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,15.270405,15.270405,1.0,1.834862,0.000000,-7.727019,-0.070890,-0.045452,-15.005164,0.0
1,KR6DS0000428,4,2023-01-13,85.0,2022-07-14,2022-12-29,2021-01-18,2024-01-17,6,2022-03-15,...,25.854626,20.854626,1.0,41.880342,1.834862,-6.039621,-0.051621,-0.028878,-16.702220,0.0
2,KR6DS0000428,5,2023-07-13,80.0,2023-01-14,2023-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,22.394745,17.394745,1.0,2.752294,41.880342,-11.280018,-0.103486,-0.101959,3.148908,0.0
3,KR6HN0000H91,3,2022-07-07,85.0,2022-01-08,2022-06-22,2021-01-08,2024-01-09,6,2022-03-09,...,17.062763,12.062763,1.0,24.770642,0.000000,-12.317089,-0.113001,-0.072526,-14.835130,0.0
4,KR6HN0000H91,4,2023-01-06,80.0,2022-07-08,2022-12-22,2021-01-08,2024-01-09,6,2022-03-09,...,17.793019,12.793019,0.0,89.830508,24.770642,-11.062150,-0.093747,-0.071109,-9.726646,0.0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   종목코드              104 non-null    object        
 1   차수                104 non-null    int64         
 2   평가종료일             104 non-null    datetime64[ns]
 3   상환조건(%)           104 non-null    float64       
 4   평가시작일             104 non-null    datetime64[ns]
 5   환매결정일             104 non-null    datetime64[ns]
 6   발행일               104 non-null    datetime64[ns]
 7   상환일               104 non-null    datetime64[ns]
 8   상환실현차수            104 non-null    int64         
 9   녹인발생일             104 non-null    datetime64[ns]
 10  기준가1              104 non-null    float64       
 11  하한 수준(%)          104 non-null    float64       
 12  녹인가1              104 non-null    float64       
 13  기초자산개수            104 non-null    int64         
 14  평가기준가             104 non-

In [5]:
long = pd.read_csv('장기피처_합치기_is_2021.csv', index_col=0)
# long.rename(columns= {'Unnamed: 0':'환매결정일'}, inplace= True)
long.index= pd.to_datetime(long.index)
# long.set_index('환매결정일', inplace= True)
long.head()

,변동,Dubai,Brent,WTI,Oman,Dubai(%),Brent(%),WTI(%),Oman(%),IIP(%),...,CLI_IDN_pct,CLI_ITA_pct,CLI_JPN_pct,CLI_KOR_pct,CLI_MEX_pct,CLI_TUR_pct,CLI_USA_pct,CLI_ZAF_pct,MSCI_China(USD),msci_China(USD)(%)
2021-01-01,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-02,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-03,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-04,0.011609,52.49,51.09,47.62,52.55,2.639812,-1.370656,-1.854905,2.536585,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-05,-0.002580,50.50,53.60,49.93,50.58,-3.791198,4.912899,4.850903,-3.748811,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271


In [6]:
short = pd.read_csv('final_total_2021.csv', index_col=0)
short.index= pd.to_datetime(short.index)

short.head()

,chn_cci_shift,kor_cci_shift,ind_cci_shift,cpi_kor_shift,cpi_chn_shift,cpi_ind_shift,cpi_jap_shift,cpi_usa_shift,chn_cci_100기준_shift,kor_cci_100기준_shift,...,일일증권평균거래량_Main Board_Other listed securities_shift,일일증권평균거래량_Main Board_Sub-total_shift,홍콩 M2 (2).1(%)_shift,홍콩직접투자(%)_shift,홍콩직접투자(%)_shift.1,홍콩파생상품(%)_shift,일일증권평균거래량_Main Board_Other equity stocks(%)_shift,홍콩거래소시가총액_Main BoardH Shares(%)_shift,국채 수익률,국채 수익률증감
2021-01-01,103.9863,99.08256,NaN,116.16,128.14,187.56,104.73,119.45,1.0,0.0,...,23635.0,140576.0,0.938086,0.49456,0.0,-4.033613,-0.373832,-1.875,0.93,0.00
2021-01-02,103.9863,99.08256,NaN,116.16,128.14,187.56,104.73,119.45,1.0,0.0,...,23635.0,140576.0,0.938086,0.49456,0.0,-4.033613,-0.373832,-1.875,0.93,0.00
2021-01-03,103.9863,99.08256,NaN,116.16,128.14,187.56,104.73,119.45,1.0,0.0,...,23635.0,140576.0,0.938086,0.49456,0.0,-4.033613,-0.373832,-1.875,0.93,0.00
2021-01-04,103.9863,99.08256,NaN,116.16,128.14,187.56,104.73,119.45,1.0,0.0,...,23635.0,140576.0,0.938086,0.49456,0.0,-4.033613,-0.373832,-1.875,0.93,0.00
2021-01-05,103.9863,99.08256,NaN,116.16,128.14,187.56,104.73,119.45,1.0,0.0,...,23635.0,140576.0,0.938086,0.49456,0.0,-4.033613,-0.373832,-1.875,0.96,0.03


In [7]:
# short.rename(columns= {'Unnamed: 0':'환매결정일'}, inplace= True)
# short['환매결정일'] = pd.to_datetime(short['환매결정일'])
# short.set_index('환매결정일', inplace= True)
# short.head()

In [8]:
# dataset.reset_index(inplace=True)

(혜선)

In [9]:
dataset.head(2)

,종목코드,차수,평가종료일,상환조건(%),평가시작일,환매결정일,발행일,상환일,상환실현차수,녹인발생일,...,환매대비상환수준(%),환매대비상환수준(%)_next,환매일종가위치_code,녹인비율(%),녹인비율_전(%),H총증감률(%),H평균증감률(%),H일평균증감률(%),H이전대비증감률(%),상환비율(%)
0,KR6DS0000428,3,2022-07-13,85.0,2022-01-14,2022-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,15.270405,15.270405,1.0,1.834862,0.000000,-7.727019,-0.070890,-0.045452,-15.005164,0.0
1,KR6DS0000428,4,2023-01-13,85.0,2022-07-14,2022-12-29,2021-01-18,2024-01-17,6,2022-03-15,...,25.854626,20.854626,1.0,41.880342,1.834862,-6.039621,-0.051621,-0.028878,-16.702220,0.0


In [10]:
long.head()

,변동,Dubai,Brent,WTI,Oman,Dubai(%),Brent(%),WTI(%),Oman(%),IIP(%),...,CLI_IDN_pct,CLI_ITA_pct,CLI_JPN_pct,CLI_KOR_pct,CLI_MEX_pct,CLI_TUR_pct,CLI_USA_pct,CLI_ZAF_pct,MSCI_China(USD),msci_China(USD)(%)
2021-01-01,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-02,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-03,0.001290,51.14,51.80,48.52,51.25,0.000000,0.000000,0.000000,0.000000,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-04,0.011609,52.49,51.09,47.62,52.55,2.639812,-1.370656,-1.854905,2.536585,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271
2021-01-05,-0.002580,50.50,53.60,49.93,50.58,-3.791198,4.912899,4.850903,-3.748811,1.800635,...,0.243702,0.239172,0.292066,0.372476,1.262923,-0.186024,0.420304,0.237433,105.505,2.757271


In [11]:
cols_long = long.columns
cols_long_diff = [col+'_diff' for col in cols_long] # 환매결정일 - 평가시작일
cols_long_std = [col+'_std' for col in cols_long] # (평가시작일 ~ 환매결정일) 표준편차

cols_long[:5], cols_long_diff[:5], cols_long_std[:5] 

(Index(['변동', 'Dubai', 'Brent', 'WTI', 'Oman'], dtype='object'),
 ['변동_diff', 'Dubai_diff', 'Brent_diff', 'WTI_diff', 'Oman_diff'],
 ['변동_std', 'Dubai_std', 'Brent_std', 'WTI_std', 'Oman_std'])

In [12]:
cols_short = short.columns
cols_short_diff = [col+'_diff' for col in cols_short] # 환매결정일 - 평가시작일
cols_short_std = [col+'_std' for col in cols_short] # (평가시작일 ~ 환매결정일) 표준편차

cols_short[:5], cols_short_diff[:5], cols_short_std[:5] 

(Index(['chn_cci_shift', 'kor_cci_shift', 'ind_cci_shift', 'cpi_kor_shift',
        'cpi_chn_shift'],
       dtype='object'),
 ['chn_cci_shift_diff',
  'kor_cci_shift_diff',
  'ind_cci_shift_diff',
  'cpi_kor_shift_diff',
  'cpi_chn_shift_diff'],
 ['chn_cci_shift_std',
  'kor_cci_shift_std',
  'ind_cci_shift_std',
  'cpi_kor_shift_std',
  'cpi_chn_shift_std'])

In [13]:
for idx in dataset.index:
    date_start = dataset.loc[idx, '평가시작일']
    date_end = dataset.loc[idx, '환매결정일']

    data_long = long.loc[date_start:date_end]
    dataset.loc[idx, cols_long_diff] = data_long.iloc[[0, -1]].diff().iloc[-1].values
    dataset.loc[idx, cols_long_std] = data_long.std().values

    data_short = short.loc[date_start:date_end]
    dataset.loc[idx, cols_short_diff] = data_short.iloc[[0, -1]].diff().iloc[-1].values
    dataset.loc[idx, cols_short_std] = data_short.std().values

In [14]:
dataset.head()

,종목코드,차수,평가종료일,상환조건(%),평가시작일,환매결정일,발행일,상환일,상환실현차수,녹인발생일,...,일일증권평균거래량_Main Board_Other listed securities_shift_std,일일증권평균거래량_Main Board_Sub-total_shift_std,홍콩 M2 (2).1(%)_shift_std,홍콩직접투자(%)_shift_std,홍콩직접투자(%)_shift.1_std,홍콩파생상품(%)_shift_std,일일증권평균거래량_Main Board_Other equity stocks(%)_shift_std,홍콩거래소시가총액_Main BoardH Shares(%)_shift_std,국채 수익률_std,국채 수익률증감_std
0,KR6DS0000428,3,2022-07-13,85.0,2022-01-14,2022-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,3133.757780,10107.556816,0.546108,0.294667,0.664902,2.220174,0.554239,1.305614,0.520532,0.072308
1,KR6DS0000428,4,2023-01-13,85.0,2022-07-14,2022-12-29,2021-01-18,2024-01-17,6,2022-03-15,...,2633.723490,16040.041616,0.224847,0.536991,1.663286,4.392289,0.000167,2.603808,0.457833,0.075119
2,KR6DS0000428,5,2023-07-13,80.0,2023-01-14,2023-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,194.578097,291.116844,0.354392,0.872565,1.801189,3.565062,0.775902,2.300384,0.180508,0.077584
3,KR6HN0000H91,3,2022-07-07,85.0,2022-01-08,2022-06-22,2021-01-08,2024-01-09,6,2022-03-09,...,3141.978085,10134.070418,0.547540,0.295440,0.666647,2.225998,0.555693,1.309039,0.521490,0.072024
4,KR6HN0000H91,4,2023-01-06,80.0,2022-07-08,2022-12-22,2021-01-08,2024-01-09,6,2022-03-09,...,2639.179947,16073.272815,0.225313,0.538104,1.666732,4.401389,0.000167,2.609203,0.462703,0.075594


In [15]:
for idx in dataset.index:
    date_end = dataset.loc[idx, '환매결정일']
    dataset.loc[idx, cols_long] = long.loc[date_end]
    dataset.loc[idx, cols_short] = short.loc[date_end]

In [16]:
dataset.head()

,종목코드,차수,평가종료일,상환조건(%),평가시작일,환매결정일,발행일,상환일,상환실현차수,녹인발생일,...,일일증권평균거래량_Main Board_Other listed securities_shift,일일증권평균거래량_Main Board_Sub-total_shift,홍콩 M2 (2).1(%)_shift,홍콩직접투자(%)_shift,홍콩직접투자(%)_shift.1,홍콩파생상품(%)_shift,일일증권평균거래량_Main Board_Other equity stocks(%)_shift,홍콩거래소시가총액_Main BoardH Shares(%)_shift,국채 수익률,국채 수익률증감
0,KR6DS0000428,3,2022-07-13,85.0,2022-01-14,2022-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,32668.0,146370.0,1.735160,0.884956,-0.853081,4.351610,1.108033,2.700270,3.20,0.00
1,KR6DS0000428,4,2023-01-13,85.0,2022-07-14,2022-12-29,2021-01-18,2024-01-17,6,2022-03-15,...,24226.0,97462.0,0.089286,0.975610,-0.933707,-6.525285,0.182315,-3.713299,3.83,-0.05
2,KR6DS0000428,5,2023-07-13,80.0,2023-01-14,2023-06-28,2021-01-18,2024-01-17,6,2022-03-15,...,28839.0,127657.0,1.154529,0.391773,2.941176,6.167401,1.187215,3.971119,3.71,-0.06
3,KR6HN0000H91,3,2022-07-07,85.0,2022-01-08,2022-06-22,2021-01-08,2024-01-09,6,2022-03-09,...,32668.0,146370.0,1.735160,0.884956,-0.853081,4.351610,1.108033,2.700270,3.16,-0.15
4,KR6HN0000H91,4,2023-01-06,80.0,2022-07-08,2022-12-22,2021-01-08,2024-01-09,6,2022-03-09,...,24226.0,97462.0,0.089286,0.975610,-0.933707,-6.525285,0.182315,-3.713299,3.67,-0.01


In [17]:
dataset.shape

(104, 491)

In [18]:
"""

# els 
# + long + long_diff + long_std
# + short + short_diff + short_std



"""
dataset.to_csv('dataset_final_240202.csv')

In [80]:
dataset.dtypes

종목코드                                                         object
차수                                                          float64
평가구분                                                         object
평가종료일                                                datetime64[ns]
상환조건(%)                                                     float64
                                                          ...      
홍콩파생상품(%)_shift                                             float64
일일증권평균거래량_Main Board_Other equity stocks(%)_shift           float64
홍콩거래소시가총액_Main BoardH Shares(%)_shift                       float64
국채 수익률                                                      float64
국채 수익률증감                                                    float64
Length: 556, dtype: object

(지운)

In [ ]:
len(dataset['환매결정일'])

In [ ]:
dataset.iloc[1749].append(short[dataset['평가시작일'][1749]:dataset['환매결정일'][1749]].iloc[-1]-short[dataset['평가시작일'][1749]:dataset['환매결정일'][1749]].iloc[0])

In [ ]:
short.columns

In [ ]:
short[dataset['평가시작일'][1749]:dataset['환매결정일'][1749]].loc[:,'chn_cci_shift'][-1]

In [ ]:
# dataset['chn_cci_shift_vol'][1749] = short[dataset['평가시작일'][1749]:dataset['환매결정일'][1749]].loc[:,'chn_cci_shift'][-1]-short[dataset['평가시작일'][1749]:dataset['환매결정일'][1749]].loc[:,'chn_cci_shift'][0]

In [ ]:

for i in (0,1749,1):
    for j in short.columns:
        Q=short[dataset['평가시작일'][i]:dataset['환매결정일'][i]].loc[:,j][-1]-short[dataset['평가시작일'][i]:dataset['환매결정일'][i]].loc[:,j][0]
        print(Q)
        

    

In [ ]:
np.std(short[dataset['평가시작일'][0]:dataset['환매결정일'][0]])

In [ ]:
cols=short.columns


In [ ]:
for col in cols:
    short[col+'_std'] = short[col].shift(1)
cols_shift = [col + '_shift' for col in cols]
